In [1]:
import os
from ultralytics import YOLO
import torch
import cv2  # Для работы с видео
from tqdm import tqdm

In [2]:
data_dir = "D:\\Documents\\pythonProjectLabelimg\\yolo_dataset\\splits"  # Папка, содержащая подпапки fold_1, fold_2, fold_3, fold_4
output_dir = "D:\\Documents\\pythonProjectLabelimg\\yolo_dataset\\res" # Папка для сохранения результатов обучения и моделей

In [3]:
# Функция для создания YAML файла (нужно реализовать)
def create_yolov8_yaml(train_dir, test_dir, yaml_path):
    """
    Создает файл YAML для YOLOv8 на основе структуры директорий.
    """
    # Получаем список классов из файлов аннотаций
    classes = ['машина', 'бордюр', 'дверь', 'стена', 'лестница', 'спуск', 'яма', 'люк', 'ограждение', 'трамвайные пути']
    num_classes = len(classes)

    # Создаем содержимое YAML файла
    yaml_content = f"""
    train: {os.path.join(train_dir, "images")}
    val: {os.path.join(test_dir, "images")}

    nc: {num_classes}
    names: {classes}
    """
    # Записываем YAML в файл
    with open(yaml_path, "w") as f:
        f.write(yaml_content)
    print(f"Created YAML file: {yaml_path}")

In [ ]:
k_folds = 4
for fold in range(8, 9):
    fold_dir = os.path.join(data_dir, f"fold_{fold}")
    train_dir = os.path.join(fold_dir, "train")
    test_dir = os.path.join(fold_dir, "test")

    map50_95_values = []
    losses = []

    save_dir = output_dir

    yaml_path = os.path.join(fold_dir, "data.yaml")
    if not os.path.exists(yaml_path):
        create_yolov8_yaml(train_dir, test_dir, yaml_path)

    print(f"Training fold {fold}/{k_folds}")

    if fold == 0:
        weights_path = 'yolov8n.pt'
    else:
        weights_path = os.path.join(save_dir, f"yolov8n_fold_{fold - 1}", "weights", "best.pt")
        print(f"Loading weights from: {weights_path}")

    model = YOLO(weights_path)

    results = model.train(
        data=yaml_path,
        epochs=70, # Настройте количество эпох
        imgsz=640,  # Настройте размер изображения
        device="cpu",  # Используем GPU, если доступен
        project = save_dir,
        name=f"yolov8n_fold_{fold}",
        save=True,
        optimizer='AdamW',
        patience=10,
        lr0=0.01,
        weight_decay=0.0005,
        batch = 32)


    best_weights_path = os.path.join(save_dir, f"yolov8n_fold_{fold}", "weights", "best.pt")
    print(f"Evaluating on Fold {fold}...")
    
    folder_path = f"D:\\Documents\\pythonProjectLabelimg\\yolo_dataset\\res\\metrics_{fold}"
    filename = f'metrics.txt'
    filepath = os.path.join(folder_path, filename)

    #if os.path.exists(filepath):
    #    os.remove(filepath)
    metrics = model.val(data=f"D:\\Documents\\pythonProjectLabelimg\\yolo_dataset\\splits\\fold_{fold}\\data.yaml")
    try:
    # Создаем и открываем файл для записи
    
        file_descriptor = os.open(filepath, os.O_WRONLY | os.O_CREAT) # os.O_WRONLY - только для записи, os.O_CREAT - создать файл, если его нет

        metrics_string = f"  Precision:  {metrics.results_dict['metrics/precision(B)']}\n" \
                     f"  Recall:{metrics.results_dict['metrics/recall(B)']}\n" \
                     f"  mAP@0.5: {metrics.results_dict['metrics/mAP50(B)']}\n" \
                     f"  mAP@0.5:0.95: {metrics.results_dict['metrics/mAP50-95(B)']}\n"
        try:
            print(f"Attempting to open file: {filepath}")  # Отладочное сообщение
            file_descriptor = os.open(filepath, os.O_WRONLY | os.O_CREAT)
            print(f"File opened successfully. Descriptor: {file_descriptor}") # Отладочное сообщение
    
            metrics_string = f"Metrics Results:\nPrecision: {metrics.box.precision}\n..."
            print(f"Metrics string: {metrics_string}") # Отладочное сообщение
    
            os.write(file_descriptor, metrics_string.encode())
            print("Metrics written successfully.") # Отладочное сообщение
        except OSError as e:
            print(f"Error: {e}")
        except Exception as e:
            print(f"General Error: {e}") # Добавлен общий обработчик исключений
        # Записываем общие метрики
        os.write(file_descriptor, metrics_string.encode()) # Преобразуем строку в байты

        # Записываем метрики по классам (если нужно)
        for i, class_name in enumerate(model.names):
            class_string = f"\nClass: {class_name}\n" \
                           f"  Precision: {metrics.box.p[i]}\n" \
                           f"  Recall: {metrics.box.r[i]}\n" 
            os.write(file_descriptor, class_string.encode()) # Преобразуем строку в байты
    
        # Закрываем файл
        os.close(file_descriptor)
    
        print(f"Metrics saved to {filename}")

    except OSError as e:
        print(f"Error writing to file: {e}")
    
    model.export(format="torchscript",  # или "torchscript"
              imgsz=[640, 640],  # Размер входного изображения
              dynamic=False,    # Зависит от ваших требований
              simplify=True,   # Упрощает модель
              opset=12,         # Версия ONNX
              keras=False,       # Не для Keras
              optimizer=True,
              project = save_dir,
              verbose=False,
              device='cpu', # Выберите устройство
              half=False) # Для FP16
    print("Модель экспортирована в формат TorchScript!")